In [1]:
import os
import pandas as pd

#Lectura dataframe codi de barris Barcelona
directorio = 'data/codibarris/BarcelonaCiutat_Barris.csv'
columnes = ["codi_barri","nom_barri"]
codisbarri = pd.read_csv(directorio)
codisbarri = codisbarri[columnes]
codisbarri.to_excel('codisbarri.xlsx', index=False)

In [2]:
#Càrrega de dataset pisos turístics
pisos = 'data/pisosturistics2023/hut_comunicacio_opendata.csv'
selected_columns = ['N_EXPEDIENT','BARRI','LONGITUD_X','LATITUD_Y'] 
pisosturistics2023 = pd.read_csv(pisos, usecols=selected_columns, encoding='utf-8')
pisosturistics2023.rename(columns={'BARRI': 'nom_barri', 'N_EXPEDIENT': 'pisos_turistics'}, inplace=True)
#Eliminació valors Nuls
pisosturistics2023.dropna(inplace=True)
pisosturistics2023

,pisos_turistics,nom_barri,LONGITUD_X,LATITUD_Y
0,01-2013-0753,el Raval,2.171125,41.379035
1,02-2014-0566,la Sagrada Família,2.173113,41.406595
2,02-2014-0207,la Dreta de l'Eixample,2.174958,41.393788
3,01-2014-0073,el Raval,2.170788,41.377598
4,02-2014-1125,la Dreta de l'Eixample,2.171954,41.390836
...,...,...,...,...
9728,02-2023-0713,l'Antiga Esquerra de l'Eixample,2.148895,41.390912
9729,02-2023-0714,Sant Antoni,2.158121,41.380003
9730,04-2023-0208,les Corts,2.134189,41.385800
9731,02-2023-0883,el Fort Pienc,2.176597,41.391755


In [3]:
#Agrupar pisos turistics per barris
pisosturisticsagrupats = pisosturistics2023.groupby(['nom_barri']).agg({'pisos_turistics': 'count'}).reset_index()

In [4]:
import pandas as pd
#Funció per buscar i assignar nom de barri i assignar codi
def assignar_codi_barri(nom, df_codi_barris):
    x = df_codi_barris[df_codi_barris['nom_barri'] == nom]
    if not x.empty:
        return x['codi_barri'].iloc[0]
    else:
        return None

In [5]:
#Assignació de codi a barris a pisosturisticsagrupats
pisosturisticsagrupats['codi_barri'] = pisosturisticsagrupats['nom_barri'].apply(lambda x: assignar_codi_barri(x, codisbarri))
#Assignació manual dels codis de barris no coicidents
pisosturisticsagrupats.loc[pisosturisticsagrupats['nom_barri'] == 'Sant Pere Santa Caterina i la Ribera', 'codi_barri'] = 4
pisosturisticsagrupats.loc[pisosturisticsagrupats['nom_barri'] == 'Vallvidrera el Tibidabo i les Planes', 'codi_barri'] = 22
pisosturisticsagrupats.loc[pisosturisticsagrupats['nom_barri'] == 'el Poble Sec', 'codi_barri'] = 11

#Conversió del codi de barri a numeric.
pisosturisticsagrupats['codi_barri'] = pd.to_numeric(pisosturisticsagrupats['codi_barri'], errors='coerce').astype('Int64')

#Combinació amb dataframe codis barris per obtindre datafraem amb tots els barris
#Pisos turístics agrupats per barris
p_turistics_group = codisbarri.merge(pisosturisticsagrupats, on='codi_barri', how='left')

#Eliminacio columnes nom_barri
p_turistics_group.drop(columns=['nom_barri_x', 'nom_barri_y'], inplace=True)

#Remplacem valors Null per valors 0. Hi ha barris que no tenen pisos turistics
p_turistics_group['pisos_turistics'].fillna(0, inplace=True)
p_turistics_group['pisos_turistics'] = p_turistics_group['pisos_turistics'].astype(int)
p_turistics_group

,codi_barri,pisos_turistics
0,1,211
1,2,194
2,3,63
3,4,158
4,5,331
...,...,...
68,69,125
69,70,32
70,71,31
71,72,14


In [6]:
#Càrrega de dataset superfície habitable
superficie = 'data/superficie/2021_densitat.csv'
superficie = pd.read_csv(superficie, encoding='utf-8')
#Canvi nomenglatura columnes
superficie.rename(columns={'Codi_Barri':'codi_barri', 'Superfície Residencial (ha)': 'superficie_hA'}, inplace=True)
#Selecció columnes codi_barri i superfícies habitable
superficie = superficie[['codi_barri', 'superficie_hA']]
superficie

,codi_barri,superficie_hA
0,1,49.6
1,2,34.1
2,3,13.5
3,4,32.4
4,5,33.6
...,...,...
68,69,24.0
69,70,28.1
70,71,15.3
71,72,23.4


In [7]:
#Combinació dataframe pisos agrupats amb duperfície i craeció de columna ratio_pisos
#La nova columna es la divisio entre els pisos turístics i la superfície habitable de cada barri.
pisos_agrupats = p_turistics_group.merge(superficie, on='codi_barri', how='outer')
pisos_agrupats['ratio_pisos'] = pisos_agrupats['pisos_turistics'] / pisos_agrupats['superficie_hA']

# Substitució valors Null per 0. Barris sense pisos turistics
pisos_agrupats['ratio_pisos'].fillna(0, inplace=True)

pisos_agrupats

,codi_barri,pisos_turistics,superficie_hA,ratio_pisos
0,1,211,49.6,4.254032
1,2,194,34.1,5.689150
2,3,63,13.5,4.666667
3,4,158,32.4,4.876543
4,5,331,33.6,9.851190
...,...,...,...,...
68,69,125,24.0,5.208333
69,70,32,28.1,1.138790
70,71,31,15.3,2.026144
71,72,14,23.4,0.598291


In [8]:
#Guardar datasets a excel 
pisos_agrupats.to_excel('p_turistics_group.xlsx', index=False)
pisosturistics2023.to_excel('p_turistics_detall.xlsx', index=False)

In [9]:
def lectura_combinacio_csv(directori, columnes):
    """
    Llegeix i combina tots els arxius d'un directori, seleccionants columnes determinades.
    Paràmetres:
    directory (str): Directori arxius csv.
    selected_columns (list): Llista columnes a seleccionar
    """
    dataframes = []
    #Lectura arxius directoris
    csv_files = [os.path.join(directori, file) for file in os.listdir(directori) if file.endswith('.csv')]
    #Iterem amb dos codificacions deiferents per problemes en alguns dels arxius.
    for file in csv_files:
        try:
            df = pd.read_csv(file, usecols=columnes, encoding='utf-8')
        except UnicodeDecodeError:
            df = pd.read_csv(file, usecols=columnes, encoding='ISO-8859-1')  
        except ValueError:
            print(f"Error {file}")
            continue
        #Normalització noms columnes per probles en alguns arxius.
        df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
        dataframes.append(df)
    unio = pd.concat(dataframes, ignore_index=True)
    return unio

In [10]:
#Dataframe lloguer barcelona 2014-2023
lloguer = 'data/lloguer14_23'  # Reemplaza con la ruta de tu carpeta
columnes_seleccio = ["Any","Trimestre","Codi_Barri","Nom_Barri","Lloguer_mitja","Preu"] 
#lectura i combinació arxius amb funció lectura_combinacio_csv
lloguer = lectura_combinacio_csv(lloguer, columnes_seleccio)
#Filtratge valors categorics de lloguer mitjà mensual (€/mes)
lloguer = lloguer[lloguer['lloguer_mitja'] == "Lloguer mitjà mensual (Euros/mes)"]
#Eliminacio columna lloguer mitjà
lloguer = lloguer.drop('lloguer_mitja', axis=1)
#Conversió a nmèric columna preu
lloguer['preu'] = pd.to_numeric(lloguer['preu'], errors='coerce')
#
#lloguer_a = lloguer.groupby(['any', 'codi_barri',"trimestre"]).agg({'preu': 'mean'}).reset_index()
lloguer.rename(columns={'preu': 'lloguer_mitja(€/mes)', 'trimestre': 'Q'}, inplace=True)
lloguer

,any,Q,codi_barri,nom_barri,lloguer_mitja(€/mes)
0,2014,1,1,el Raval,589.55
1,2014,1,2,el Barri Gòtic,712.79
2,2014,1,3,la Barceloneta,540.71
3,2014,1,4,"Sant Pere, Santa Caterina i la Ribera",673.44
4,2014,1,5,el Fort Pienc,736.09
...,...,...,...,...,...
5324,2023,2,69,Diagonal Mar i el Front Marítim del Poblenou,1400.60
5325,2023,2,70,el Besòs i el Maresme,746.90
5326,2023,2,71,Provençals del Poblenou,1204.20
5327,2023,2,72,Sant Martí de Provençals,960.90


In [11]:
nuls = lloguer[lloguer.isna().any(axis=1)]
#VAlors Nuls
print(nuls)

       any  Q  codi_barri                   nom_barri  lloguer_mitja(€/mes)
11    2014  1          12  la Marina del Prat Vermell                   NaN
41    2014  1          42                    la Clota                   NaN
53    2014  1          54                  Torre Baró                   NaN
55    2014  1          56                    Vallbona                   NaN
57    2014  1          58               Baró de Viver                   NaN
...    ... ..         ...                         ...                   ...
5297  2023  2          42                    la Clota                   NaN
5302  2023  2          47                 Can Peguera                   NaN
5309  2023  2          54                  Torre Baró                   NaN
5311  2023  2          56                    Vallbona                   NaN
5313  2023  2          58               Baró de Viver                   NaN

[184 rows x 5 columns]


In [12]:
#S'assigna el valor de 0 als valors Null. Hi ha barris sense informació, però son barris industrials i no 
#son determinats pel la pregunta que ha de respondre la visualització.
lloguer.fillna(0, inplace=True)
#Guardem dataframe en arxiu excel.
lloguer.to_excel('evolucio_lloguer.xlsx', index=False)

In [13]:
#Creació dataframe amb increment del preu del lloguer de 2014 a 2023, per a cada barri
df_2014 = lloguer[(lloguer['any'] == 2014) & (lloguer['Q'] == 1)] 
df_2023 = lloguer[(lloguer['any'] == 2023) & (lloguer['Q'] == 2)] #Dades fins al 2ª T de 2023


In [14]:
#Definim codi barri com a index per creació de dataframe
df_2014_index = df_2014.set_index('codi_barri')
df_2023_index = df_2023.set_index('codi_barri')

# Calcul increment percentual del preu del lloguer de 2014 fins a 2023.
increment = ((df_2023_index['lloguer_mitja(€/mes)'] - df_2014_index['lloguer_mitja(€/mes)'])/df_2014_index['lloguer_mitja(€/mes)'])*100
increment = increment.to_frame(name='increment_preu_lloguer')

#S'afegeix la columna nom_barri
increment = increment.merge(df_2014_index[['nom_barri']], left_index=True, right_index=True)

#Ressent index codi_barri per convertirlo en columna i ordenem dataframe
increment.reset_index(inplace=True)
increment = increment[['codi_barri', 'nom_barri', 'increment_preu_lloguer']]

#Substituim valors nulls per 0. Barris que no hi ha informació.
increment.fillna(0, inplace=True)

increment

,codi_barri,nom_barri,increment_preu_lloguer
0,1,el Raval,53.574760
1,2,el Barri Gòtic,81.526116
2,3,la Barceloneta,85.348893
3,4,"Sant Pere, Santa Caterina i la Ribera",64.454146
4,5,el Fort Pienc,61.040090
...,...,...,...
68,69,Diagonal Mar i el Front Marítim del Poblenou,61.601477
69,70,el Besòs i el Maresme,44.496034
70,71,Provençals del Poblenou,89.428976
71,72,Sant Martí de Provençals,49.286891


In [15]:
increment.to_excel('increment_lloguer.xlsx', index=False)

In [16]:
#Dataframe renda bruta mitja per persona barcelona 2021
renda = 'data/renda/2021_atles_renda_bruta_persona.csv' 
#Lectura arxiu renda mitjana 2021
renda = pd.read_csv(renda, encoding='utf-8')
#Selecció de columnes
renda = renda[['Codi_Barri','Nom_Barri','Seccio_Censal','Import_Renda_Bruta_€']]
#Agrupació renda mitjana pere barri
renda_p_2021 = renda.groupby(['Codi_Barri', 'Nom_Barri']).agg({'Import_Renda_Bruta_€': 'mean'}).reset_index()
#Renombrar columnes
renda_p_2021.rename(columns={'Codi_Barri': 'codi_barri', 'Nom_Barri':'nom_barri', 'Import_Renda_Bruta_€': 'renda_p'}, inplace=True)
renda_p_2021

,codi_barri,nom_barri,renda_p
0,1,el Raval,12133.380952
1,2,el Barri Gòtic,16996.555556
2,3,la Barceloneta,15986.545455
3,4,"Sant Pere, Santa Caterina i la Ribera",18077.538462
4,5,el Fort Pienc,23084.850000
...,...,...,...
68,69,Diagonal Mar i el Front Marítim del Poblenou,25008.285714
69,70,el Besòs i el Maresme,13337.230769
70,71,Provençals del Poblenou,19156.000000
71,72,Sant Martí de Provençals,18040.666667


In [17]:
#Creem categoria tipus_barri, categoritzant renda_p en barri pobre, barri ric, barri renda mitja
#Apliquem es talls en els percentis 33 i 80.
p_33 = renda_p_2021['renda_p'].quantile(0.33)
p_80 = renda_p_2021['renda_p'].quantile(0.80)
#Definim els punts de tall
percentils = [0, p_33, p_80, float('inf')]
categories = ['barri pobre', 'barri renda mitja', 'barri ric']

#Creem variable tipus_barri
renda_p_2021['tipus_barri'] = pd.cut(renda_p_2021['renda_p'], bins=percentils, labels=categories, include_lowest=True)
renda_p_2021

,codi_barri,nom_barri,renda_p,tipus_barri
0,1,el Raval,12133.380952,barri pobre
1,2,el Barri Gòtic,16996.555556,barri pobre
2,3,la Barceloneta,15986.545455,barri pobre
3,4,"Sant Pere, Santa Caterina i la Ribera",18077.538462,barri renda mitja
4,5,el Fort Pienc,23084.850000,barri renda mitja
...,...,...,...,...
68,69,Diagonal Mar i el Front Marítim del Poblenou,25008.285714,barri renda mitja
69,70,el Besòs i el Maresme,13337.230769,barri pobre
70,71,Provençals del Poblenou,19156.000000,barri renda mitja
71,72,Sant Martí de Provençals,18040.666667,barri renda mitja


In [18]:
#Guardar dataframen en arxiu xlxs.
renda_p_2021.to_excel('renda_p.xlsx', index=False)

In [20]:
#Cargar arxius cnavis de domicili
#Dataframe lloguer barcelona 2014-2023
canvis_domicili = 'data/canvidomicili'  # Reemplaza con la ruta de tu carpeta
columnes_seleccio = ['Any','Codi_Barri','Nom_Barri','Valor','CODI_BARRI_DEST','SEXE'] 
#lectura i combinació arxius amb funció lectura_combinacio_csv
canvis_domicili = lectura_combinacio_csv(canvis_domicili, columnes_seleccio)
#Agragar nom de barri destí. Creació diccionari amb df codibarris
nom_barri = codisbarri.set_index('codi_barri')['nom_barri'].to_dict()
#Incorporem nom barri a partir de diccionari.
canvis_domicili['nom_barri_dest'] = canvis_domicili['codi_barri_dest'].map(nom_barri)
#Ordenació columnes
canvis_domicili = canvis_domicili.reindex(columns=['any', 'codi_barri', 'nom_barri', 'codi_barri_dest', 'nom_barri_dest','sexe', 'valor'])
#Canvi nomenglatura variable valors a canvi_domicili
canvis_domicili.rename(columns={'valor': 'canvi_domicili'}, inplace=True)
#Conversió canvi domicili a numèric
canvis_domicili['canvi_domicili'] = pd.to_numeric(canvis_domicili['canvi_domicili'], errors='coerce').fillna(0).astype(int)
#Guardar dataframe en arxiu xlxs.
canvis_domicili.to_excel('canvisdocimicili2018_22.xlsx', index=False)
canvis_domicili

,any,codi_barri,nom_barri,codi_barri_dest,nom_barri_dest,sexe,canvi_domicili
0,2018,1,el Raval,1,el Raval,1,889
1,2018,1,el Raval,1,el Raval,2,1371
2,2018,1,el Raval,2,el Barri Gòtic,1,118
3,2018,1,el Raval,2,el Barri Gòtic,2,450
4,2018,1,el Raval,3,la Barceloneta,1,34
...,...,...,...,...,...,...,...
39328,2022,73,la Verneda i la Pau,71,Provençals del Poblenou,2,18
39329,2022,73,la Verneda i la Pau,72,Sant Martí de Provençals,1,67
39330,2022,73,la Verneda i la Pau,72,Sant Martí de Provençals,2,68
39331,2022,73,la Verneda i la Pau,73,la Verneda i la Pau,1,210


In [21]:
#Acumular canvis de docimili dels últims 5 anys
canvis_docmicili_acumulats = canvis_domicili.groupby(['codi_barri','nom_barri','codi_barri_dest','nom_barri_dest','sexe'])['canvi_domicili'].sum().reset_index()
canvis_docmicili_acumulats.to_excel('canvisdocimiciliacumulats.xlsx', index=False)
canvis_docmicili_acumulats

,codi_barri,nom_barri,codi_barri_dest,nom_barri_dest,sexe,canvi_domicili
0,1,el Raval,1,el Raval,1,4319
1,1,el Raval,1,el Raval,2,5921
2,1,el Raval,2,el Barri Gòtic,1,735
3,1,el Raval,2,el Barri Gòtic,2,2183
4,1,el Raval,3,la Barceloneta,1,193
...,...,...,...,...,...,...
9793,73,la Verneda i la Pau,71,Provençals del Poblenou,2,113
9794,73,la Verneda i la Pau,72,Sant Martí de Provençals,1,353
9795,73,la Verneda i la Pau,72,Sant Martí de Provençals,2,300
9796,73,la Verneda i la Pau,73,la Verneda i la Pau,1,1035
